In [1]:
from EventStreamData import *
from GUID import *
import pandas as pd 
import numpy as np 
from pandas.io.json import json_normalize

In [3]:
gameresult = GameResult()
display(pd.DataFrame(gameresult.df_raw.head(5)))

,esports_match_id,start_time,num_map,map_name,map_type,map_winner,team_one_name,team_two_name
0,31054,1586031115938,1,576460752303425122,CONTROL,7695,7695,7697
1,31054,1586031884822,2,576460752303423963,ASSAULT,7697,7697,7695
2,31054,1586034492985,3,576460752303424955,PAYLOAD,7695,7697,7695
3,31054,1586035710426,4,576460752303425165,HYBRID,7695,7697,7695
4,31071,1586037937586,1,576460752303425182,CONTROL,4406,4406,4523
